#### 1. Data Processing: 

###### a) Use the "ImageDataGenerator()" class from keras.processing.image to build out an instance called "train_datagen" 

In [42]:
import keras

In [43]:
from keras.preprocessing.image import ImageDataGenerator

In [44]:
train_datagen=ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)

###### b) Then build your training set by using the method ".flow_from_directory()"

In [45]:
train_generator = train_datagen.flow_from_directory(
        'dataset_train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='categorical')

# Pulling the data 

Found 88 images belonging to 4 classes.


###### c) Take a look at your training set: 
* What is the image shape of each training observation?
Category 1: It shape like letter 'n'
Category 2: It shape like a windown
Category 3: It shape like a hat
Category 4: It shape like a cross
* How many total classes do we need to predict on? 
I think we need 4 class for this project

#### 2. Initial Classifier Build: Now use keras to build an initial image classifier with the following specifications.

In [46]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D

model= Sequential()
model.add(Conv2D(filters = 32,kernel_size = 3,input_shape=(64, 64,3),activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Conv2D(filters = 64,kernel_size = 3,activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Flatten())
model.add(Dense(128,activation = 'relu'))
model.add(Dense(4,activation = 'softmax'))
# Compile model
model.compile(optimizer = 'adam',loss ='categorical_crossentropy',metrics=['accuracy'])

#### 3. Model Runs: This will be run various times with different numbers of steps_per_epoch and epochs. 

###### a) Use .fit_generator() with the training set.

In [47]:
# Fit the model
model.fit_generator(train_generator,steps_per_epoch = 10,epochs = 10)

Epoch 1/10
10/10 [==============================] - 3s 333ms/step - loss: 1.3352 - accuracy: 0.5068
Epoch 2/10
10/10 [==============================] - 3s 331ms/step - loss: 0.4110 - accuracy: 0.9062
Epoch 3/10
10/10 [==============================] - 4s 375ms/step - loss: 0.1646 - accuracy: 0.9459
Epoch 4/10
10/10 [==============================] - 3s 325ms/step - loss: 0.0781 - accuracy: 0.9764
Epoch 5/10
10/10 [==============================] - 3s 338ms/step - loss: 0.0435 - accuracy: 0.9831
Epoch 6/10
10/10 [==============================] - 3s 311ms/step - loss: 0.0421 - accuracy: 0.9931
Epoch 7/10
10/10 [==============================] - 4s 351ms/step - loss: 0.0206 - accuracy: 0.9932
Epoch 8/10
10/10 [==============================] - 4s 392ms/step - loss: 0.0140 - accuracy: 0.9932
Epoch 9/10
10/10 [==============================] - 3s 328ms/step - loss: 0.0056 - accuracy: 1.0000
Epoch 10/10
10/10 [==============================] - 3s 332ms/step - loss: 0.0064 - accuracy: 0.9966

###### b) Write out each model & model_weights to a file



In [48]:
# write model and model weights to disk
model_yaml = model.to_yaml()
with open("model_1.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

model.save_weights("model_1.h7")
print("Saved model to disk")

Saved model to disk


###### c) Predict using the model built in step 2

In [49]:
import os, glob
import numpy as np
from keras.models import model_from_yaml
from keras.preprocessing import image

# load model from disk
yaml_file = open('model_1.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
model = model_from_yaml(loaded_model_yaml)
# load weights into new model
model.load_weights("model_1.h7")
print("Loaded model from disk")

Loaded model from disk


In [50]:
# test data path
img_dir = "dataset_test" # Enter Directory of all images

# iterate over each test image
# make a prediction and add to results 
data_path = os.path.join(img_dir, '*g')
files = glob.glob(data_path)
data = []
results = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result = model.predict(img)
    r = np.argmax(result, axis=1)
    results.append(r[0])
results

[3, 0, 0, 0, 1, 2, 1, 1]

###### d) Determine accuracy.


In [51]:
files

['dataset_test/C033.png',
 'dataset_test/1022.png',
 'dataset_test/4011.png',
 'dataset_test/1053.png',
 'dataset_test/6051.png',
 'dataset_test/4053.png',
 'dataset_test/C014.png',
 'dataset_test/6023.png']

In [52]:
actual_label=[3,0,2,0,1,2,3,1]

##### The accurate lable is 3,0,2,0,1,2,3,1


In [53]:
from sklearn.metrics import accuracy_score

In [54]:
accuracy_score(actual_label,results)

0.75

###### e) Run this process for the following combinations:


* (steps_per_epoch: 10, epochs: 10) <- the one we just did 
* (steps_per_epoch: 10, epochs: 20)
* (steps_per_epoch: 10, epochs: 30)
* (steps_per_epoch: 30, epochs: 10)
* (steps_per_epoch: 30, epochs: 20)
* (steps_per_epoch: 30, epochs: 30)
* (steps_per_epoch: 50, epochs: 10)
* (steps_per_epoch: 50, epochs: 20)
* (steps_per_epoch: 50, epochs: 30)
* (steps_per_epoch: 50, epochs: 100)

In [59]:
import pandas as pd
result_df=pd.DataFrame(columns=['Steps per Epoch','Epochs','Accuracy'])
steps_per_epoch=[10,10,10,30,30,30,50,50,50,50]
epochs=[10,20,30,10,20,30,10,20,30,100]

for i in range(10):
    # Compile model
    model.compile(optimizer = 'adam',loss ='categorical_crossentropy',metrics=['accuracy'])
    # Fit the model
    model.fit_generator(train_generator,steps_per_epoch = steps_per_epoch[i],epochs= epochs[i])
    # write model and model weights to disk
    model_yaml = model.to_yaml()
    with open("model_1.yaml", "w") as yaml_file:
        yaml_file.write(model_yaml)

    model.save_weights("model_"+str(steps_per_epoch[i])+str(epochs[i])+".h5")
    #print("Saved model to disk")

    # load model from disk
    yaml_file = open('model_1.yaml', 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    model = model_from_yaml(loaded_model_yaml)
    # load weights into new model
    model.load_weights("model_"+str(steps_per_epoch[i])+str(epochs[i])+".h5")
    #print("Loaded model from disk")
    # test data path
    img_dir = "dataset_test" # Enter Directory of all images

    # iterate over each test image
    # make a prediction and add to results 
    data_path = os.path.join(img_dir, '*g')
    files = glob.glob(data_path)
    data = []
    results = []
    for f1 in files:
        img = image.load_img(f1, target_size = (64, 64))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis = 0)
        data.append(img)
        result = model.predict(img)
        r = np.argmax(result, axis=1)
        results.append(r)
        
    result_df.loc[i]=[steps_per_epoch[i],epochs[i],accuracy_score(actual_label,results)]

Epoch 1/10
10/10 [==============================] - 3s 327ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 2/10
10/10 [==============================] - 3s 303ms/step - loss: 0.0057 - accuracy: 0.9965
Epoch 3/10
10/10 [==============================] - 3s 327ms/step - loss: 2.0336e-04 - accuracy: 1.0000
Epoch 4/10
10/10 [==============================] - 4s 382ms/step - loss: 5.1141e-04 - accuracy: 1.0000
Epoch 5/10
10/10 [==============================] - 4s 363ms/step - loss: 7.6821e-05 - accuracy: 1.0000
Epoch 6/10
10/10 [==============================] - 3s 325ms/step - loss: 1.3053e-04 - accuracy: 1.0000
Epoch 7/10
10/10 [==============================] - 4s 363ms/step - loss: 1.3342e-04 - accuracy: 1.0000
Epoch 8/10
10/10 [==============================] - 4s 371ms/step - loss: 1.9374e-05 - accuracy: 1.0000
Epoch 9/10
10/10 [==============================] - 4s 352ms/step - loss: 0.0049 - accuracy: 0.9965
Epoch 10/10
10/10 [==============================] - 4s 378ms/step - loss: 0

50/50 [==============================] - 19s 383ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/30
50/50 [==============================] - 18s 369ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/30
50/50 [==============================] - 19s 378ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/30
50/50 [==============================] - 18s 369ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/30
50/50 [==============================] - 18s 359ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 11/30
50/50 [==============================] - 17s 333ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 12/30
50/50 [==============================] - 19s 376ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 13/30
50/50 [==============================] - 17s 349ms/step - loss: 1.7361e-10 - accuracy: 1.0000
Epoch 14/30
50/50 [==============================] - 20s 409ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 15/30
50/50 [==============================] - 20s 402m

50/50 [==============================] - 17s 342ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 54/100
50/50 [==============================] - 16s 329ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 55/100
50/50 [==============================] - 17s 348ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 56/100
50/50 [==============================] - 17s 333ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 57/100
50/50 [==============================] - 20s 390ms/step - loss: 7.4642e-11 - accuracy: 1.0000
Epoch 58/100
50/50 [==============================] - 21s 412ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 59/100
50/50 [==============================] - 18s 360ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 60/100
50/50 [==============================] - 17s 346ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 61/100
50/50 [==============================] - 17s 333ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 62/100
50/50 [==============================

###### f) Create a final dataframe that combines the accuracy across each combination.





In [61]:
result_df

,Steps per Epoch,Epochs,Accuracy
0,10.0,10.0,0.750
1,10.0,20.0,0.750
2,10.0,30.0,0.750
3,30.0,10.0,0.625
4,30.0,20.0,0.750
5,30.0,30.0,0.750
6,50.0,10.0,0.750
7,50.0,20.0,0.750
8,50.0,30.0,0.750
9,50.0,100.0,0.750


###### 5. Name two uses of zero padding in CNN.
(1) control the shrinkage of dimension after applying filters larger than 1x1
(2) avoid loosing information at the boundaries
###### 6. What is the use of a 1 x 1 kernel in CNN? 
the 1X1 Convolution layer was used for ‘Cross Channel Down sampling’ or Cross Channel Pooling. In other words, 1X1 Conv was used to reduce the number of channels while introducing non-linearity.This effect of cross channel down-sampling is called ‘Dimensionality reduction’.

###### 7. What are the advantages of a CNN over a fully connected DNN for this image classification problem?
Use CNN over a fully connected DNN helps to reduce the number of parameters, thus shorten the running time of the program.CNN is more efficient in terms of memory and complexity. Imagine NNs and CNNs with billions of neurons, then CNNs would be less complex and saves memory compared to the NN.
CNN have the abaility of capturing and able to learn relevant features from an image, this feature learning function helps the image classification a lot. 
Also for this problem, CNNs are very good feature extractors. This means that you can extract useful attributes from an already trained CNN with its trained weights by feeding your data on each level and tune the CNN a bit for the specific task. 